In [1]:
# Biblioteker
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

In [41]:
# Konstanter
g = 9.81
m = 0.0301
c = 2/5

In [42]:
# Utregning av bane
festepunkter_x = np.arange(0, 1400+200, 200)
festepunkter_y = np.array([300, 247, 159, 159, 195, 190, 131, 152,])

x = np.arange(0, 1400+1, 1)

cs = CubicSpline(festepunkter_x,festepunkter_y,bc_type='natural')
y = cs(x)
dy = cs(x,1)
ddy = cs(x,2)

In [43]:
# Plot av banen y(x)
baneform = plt.figure('y(x)',figsize=(12,6))
plt.title('Banens form', fontsize=20)
plt.plot(x,y,festepunkter_x,festepunkter_y,'o')
plt.ylim(0,300)
plt.xlim(-50,1450)
plt.grid()

for i, j in zip(festepunkter_x, festepunkter_y):
   strlen = len(f"({str(i)}, {str(j)})")
   plt.text(i-strlen*6, j+10, '({}, {})'.format(i, j))

plt.show()

In [44]:
# Plot av derivert av bane
baneform = plt.figure('y(x)',figsize=(12,6))
plt.title('Banens derivert', fontsize=20)
plt.plot(x,dy)
plt.grid()
plt.show()

In [45]:
# Plot av kulas fart
def kfart(banehøyde):
    y0 = y[0] / 1000 # m
    return np.sqrt(2*g*(y0-(banehøyde/1000))/(1+c))

baneform = plt.figure('y(x)',figsize=(12,6))
plt.title('Kulens fart', fontsize=20)
plt.plot(x, kfart(y))

plt.grid()
plt.show()

In [46]:
# Plot av helningsvinkel
def helning(dy):
    return np.arctan(dy)

baneform = plt.figure('y(x)',figsize=(12,6))
plt.title('Helningsvinkel', fontsize=20)
plt.plot(x, helning(dy))

plt.grid()
plt.show()

In [47]:
# Tidsutvikling
def dt(n):
    dx = 0.001 # m
    fart = kfart(y[n])
    fart_next = kfart(y[n+1])
    return (2 * dx) / ((fart + fart_next) * abs(np.cos(helning(dy[n]))))
dt_arr = [dt(i) for i in range(0, len(x)-1)]

In [48]:
# Total tid
tid = sum(dt_arr)

In [49]:
# Plot av tidsutvikling
baneform = plt.figure('y(x)',figsize=(12,6))
plt.title('Tidsutvikling', fontsize=20)
plt.plot(x[:-1], dt_arr)
plt.grid()

plt.show()

In [50]:
# Utregning av kinetisk energi i sluttpunktet
E = (1+c)*m*kfart(y[-1]/1000)**2/2
print(E)

In [11]:
def Normalkraft(n):
    m = 0.031
    c = 2/5
    g = 9.81
    y_0 = y[0]
    sentrip = (2*g(y_0 - y[n])/(1+c)) * ddy[n]/(1+(dy[n]**2))**(3/2)
    return m(g*np.cos(helning[n])+sentrip)

In [12]:
#Friksjonskraft
def F(n): 
    m = 0.031
    g = 9.81
    f = 2*m*g*np.sin(helning[n])/7
    if abs(f) > 0.4 * Normalkraft(n):
        return 0.4 * Normalkraft(n)
    else:
        return f

In [13]:
#Forholdet mellom friksjonskraften og normalkraften
def aFrictionousRelationship(n):
    return F(n)/Normalkraft(n)
